## Setup

### Create Remote Connection

The CONNECTION resource is the secure, IAM-governed "handshake" between BigQuery and other Google Cloud services, most notably Vertex AI and Google Cloud Storage.

When a connection is created, it is associated with a unique service account. This service account acts as a proxy for BigQuery, and it must be granted the appropriate IAM roles to interact with external services. For instance, to call a Vertex AI model, the connection's service account needs the
Vertex AI User (roles/aiplatform.user) role.

This mechanism ensures that all interactions are authenticated, authorized, and auditable, adhering to the principle of least privilege and satisfying enterprise security requirements

In [ ]:
!bq mk --connection --location=US \
    --project_id=$GOOGLE_CLOUD_PROJECT \
    --connection_type=CLOUD_RESOURCE masterclass

In [ ]:
SERVICE_ACCT = !bq show --format=prettyjson --connection $GOOGLE_CLOUD_PROJECT.us.masterclass | grep "serviceAccountId" | cut -d '"' -f 4
SERVICE_ACCT_EMAIL = SERVICE_ACCT[-1]
print(SERVICE_ACCT_EMAIL)

In [ ]:
import os
PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))
!gcloud projects add-iam-policy-binding --format=none --condition=None $PROJECT_ID --member=serviceAccount:$SERVICE_ACCT_EMAIL --role=roles/aiplatform.user

In [ ]:
!gcloud projects get-iam-policy $PROJECT_ID --flatten=bindings --filter=bindings.members:serviceAccount:$SERVICE_ACCT_EMAIL --format='value(bindings.role)'

In [ ]:
# Restart Kernel
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Create Dataset & Register Models

The remote model is the primary abstraction layer that makes in-database AI possible. Using a standard CREATE MODEL statement with a REMOTE WITH CONNECTION clause, a data engineer can register an externally hosted model as a callable object within a BigQuery dataset.

This remote model can point to a powerful foundation model hosted on Vertex AI, such as Google's Gemini family or partner models from Anthropic and Mistral AI, or even a custom-trained model deployed on a Vertex AI Endpoint.

The significance of this abstraction cannot be overstated. It transforms what would otherwise be a complex programming task—involving setting up a development environment, using a client library, managing API keys, and handling HTTP requests—into a simple, familiar SQL function call like ML.GENERATE_TEXT or ML.PREDICT.

![GenAI Workflow](https://cloud.google.com/static/bigquery/images/gen-ai-workflow.png)

The data engineer operates entirely within the BigQuery environment, while the platform handles the underlying mechanics of invoking the model, passing the data, and returning the results. This is the core mechanism that "brings the model to the data," eliminating the need for complex MLOps pipelines for a wide range of use cases.

In [ ]:
%%bigquery
CREATE SCHEMA IF NOT EXISTS masterclass
OPTIONS (
    description = 'Data Lakehouse Mastery masterclass at the Google Cloud Summit Switzerland 2025 in Zurich',
    location = 'US');

In [ ]:
%%bigquery

CREATE MODEL IF NOT EXISTS masterclass.gemini_flash
  REMOTE WITH CONNECTION `us.masterclass`
  OPTIONS (ENDPOINT='gemini-2.0-flash');

In [ ]:
%%bigquery
CREATE MODEL IF NOT EXISTS `masterclass.text_multilingual_embedding`
REMOTE WITH CONNECTION `us.masterclass`
OPTIONS (endpoint = 'text-multilingual-embedding-002')

In [ ]:
%%bigquery
CREATE MODEL IF NOT EXISTS `masterclass.text_embedding`
REMOTE WITH CONNECTION `us.masterclass`
OPTIONS (endpoint = 'text-embedding-005')

## Creating Semantic Representations with ML.GENERATE_EMBEDDING

### Generate and Store Embeddings

Next, you will execute a CREATE TABLE AS SELECT statement. This query uses the ML.GENERATE_EMBEDDING function to process the abstract text from the patents table. The function outputs a 768-dimension vector for each abstract, which is then stored in a new table alongside the original patent ID and title for future reference.

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `masterclass.patent_embeddings`
AS
SELECT
  publication_number,
  title,
  content AS abstract,
  ml_generate_embedding_result AS embedding
FROM
  ML.GENERATE_EMBEDDING(
    MODEL `masterclass.text_embedding`,
    (
      SELECT
        publication_number,
        title,
        abstract AS content
      FROM
        `patents-public-data.google_patents_research.publications`
      WHERE
        abstract IS NOT NULL AND LENGTH(abstract) > 0
      LIMIT 10000 -- Limit for workshop purposes
    )
  );

### Create a Vector Index
You will run the CREATE VECTOR INDEX command on the embedding column of the newly created table. This command initiates an asynchronous job to build an index using an Approximate Nearest Neighbor (ANN) algorithm. The IVF (Inverted File) index type is a common choice, which works by clustering the vectors. The DISTANCE_TYPE is set to COSINE, which is effective for measuring the similarity of text embeddings.

In [ ]:
%%bigquery

CREATE OR REPLACE VECTOR INDEX `patent_embedding_index`
ON `masterclass.patent_embeddings`(embedding)
OPTIONS(index_type = 'IVF', distance_type = 'COSINE');

## Accelerating Search with VECTOR_SEARCH

With embeddings created, the next step is to enable efficient searching. A brute-force search comparing a query vector to millions of document vectors would be computationally expensive. Vector indexes solve this problem.


### Perform Semantic Search
Now, you can perform a semantic search using the VECTOR_SEARCH function. This query first generates an embedding for a user's natural language question (e.g., "inventions related to solar panel efficiency"). It then uses this query vector to search the indexed patent table, efficiently finding the top_k most semantically similar patent abstracts based on cosine similarity.

In [ ]:
SEARCH_QUERY = "inventions related to solar panel efficiency"  # @param {type:"string"}
BQ_PARAMS = {"search_query": SEARCH_QUERY}

print(f"BQ_PARAMS: {BQ_PARAMS}")

In [ ]:
%%bigquery --params $BQ_PARAMS
SELECT
  base.publication_number,
  base.title,
  base.abstract,
  distance
FROM
  VECTOR_SEARCH(
    TABLE `masterclass.patent_embeddings`,
    'embedding',
    (
      SELECT ml_generate_embedding_result AS embedding
      FROM ML.GENERATE_EMBEDDING(
        MODEL `masterclass.text_embedding`,
        (SELECT @search_query AS content)
      )
    ),
    top_k => 5
  );


### Construct the RAG Query
The final task is to construct a single, multi-step SQL query using Common Table Expressions (CTEs). This query orchestrates the entire RAG workflow:

1. A CTE (retrieved_context) uses VECTOR_SEARCH to find the top 5 relevant patent abstracts for a given question, just as in the previous step.
2. A second CTE (prompt_construction) concatenates the user's question with the retrieved abstracts to form a single, comprehensive prompt.
3. The final SELECT statement passes this dynamically constructed prompt to the Gemini Flash model via ML.GENERATE_TEXT. The prompt explicitly instructs the model to answer the question based only on the provided context, which helps to ground the response in factual data and reduce hallucinations.


In [ ]:
%%bigquery --params $BQ_PARAMS
  SELECT
    base.title,
    base.abstract
  FROM
    VECTOR_SEARCH(
      TABLE `masterclass.patent_embeddings`,
      'embedding',
      (
        SELECT ml_generate_embedding_result AS embedding
        FROM ML.GENERATE_EMBEDDING(
          MODEL `masterclass.text_embedding`,
          (SELECT @search_query AS content)
        )
      ),
      top_k => 5
    )

In [ ]:
%%bigquery rag_result --params $BQ_PARAMS
WITH retrieved_context AS (
  SELECT
    base.title,
    base.abstract
  FROM
    VECTOR_SEARCH(
      TABLE `masterclass.patent_embeddings`,
      'embedding',
      (
        SELECT ml_generate_embedding_result AS embedding
        FROM ML.GENERATE_EMBEDDING(
          MODEL `masterclass.text_embedding`,
          (SELECT @search_query AS content)
        )
      ),
      top_k => 5
    )
),
prompt_construction AS (
  SELECT
    CONCAT(
      """Question: What are some inventions related to solar panel efficiency?
      Answer the question based ONLY on the following context.
      Context: """,
      STRING_AGG(retrieved_context.abstract, ' ')
    ) AS prompt
  FROM retrieved_context
)
SELECT
  ml_generate_text_result['candidates'][0]['content']['parts'][0]['text'] AS generated_answer
FROM
  ML.GENERATE_TEXT(
    MODEL `masterclass.gemini_flash`,
    TABLE prompt_construction
  );

In [ ]:
from IPython.display import Markdown as md
md(rag_result['generated_answer'][0].replace("\"","").replace("\\n","\n"))

## Cleaning up

In [ ]:
#
# !bq rm -r -f $PROJECT_ID:masterclass
# !bq rm --connection --project_id=$PROJECT_ID --location=us masterclass
#